# Loading/Manipulating excel data with Pandas

In [46]:
import pandas as pd
import numpy as np

data_loc = '../data/sampling_details.xlsx'

release_date = pd.Timestamp('2005-03-15')

# load the sentinel fields sheet
sentinel_fields_data = pd.read_excel(data_loc,sheetname='Kal-sentinels-raw')
# rename the headings with spaces in them
sentinel_fields_data.rename(columns={"Field descrip":"descrip","date emerged":"date", 
                                     "Field ID (jpgs)": "id",
                                     "Field ID (paper)":"paperid"}, inplace=True)
sentinel_fields_data.drop('descrip',1,inplace=True)
sentinel_fields_data.drop('paperid',1,inplace=True)
sentinel_fields_data.sort_values(['id','date'], inplace=True)
# get sum of all the emergences
col_list = list(sentinel_fields_data)
for name in ['id','date']:
    col_list.remove(name)
sentinel_fields_data['All_total'] = sentinel_fields_data[col_list].sum(axis=1)
# get the number of E Hayati emergences per day
sentinel_fields_data['E_total'] = sentinel_fields_data[['Efemales','Emales']].sum(axis=1)
sentinel_fields_data['datePR'] = sentinel_fields_data['date'] - release_date
print(sentinel_fields_data['datePR'].min().days)
PR29 = sentinel_fields_data['datePR'] == pd.Timedelta('29 days')
sentinel_fields_data[PR29].dtypes

19


id                    object
date          datetime64[ns]
Efemales               int64
Emales                 int64
morpho1                int64
morpho2                int64
morpho3                int64
morpho4                int64
morpho5                int64
morpho6                int64
Wfnum                  int64
All_total              int64
E_total                int64
datePR       timedelta64[ns]
dtype: object